# Featurizer #
This notebook demonstrates how to use `pyTigerGraph` for feature engineering tasks on graphs stored in `TigerGraph`.

**Note**: For some built-in graph algorithms to work, the associated UDF needs to be installed. See tutorial 2.1_UDF_installer for details.

## Connection to Database ##
The `TigerGraphConnection` class represents a connection to the TigerGraph database. Under the hood, it stores the necessary information to communicate with the database. Please see its [documentation](https://docs.tigergraph.com/pytigergraph/current/intro/) for details.

In [ ]:
from pyTigerGraph import TigerGraphConnection

In [ ]:
conn = TigerGraphConnection(
    host="http://127.0.0.1", # Change the address to your database server's
    graphname="Cora",
    username="tigergraph",
    password="tigergraph"
)

# Uncomment below to get and set token if token authentication is enabled. 
#conn.getToken(<secret>) # <secret> is your user secret. See https://docs.tigergraph.com/tigergraph-server/current/user-access/managing-credentials#_secrets for details.

In [ ]:
# Graph schema and other information.
print(conn.gsql("ls"))

In [ ]:
# Number of vertices for every vertex type
conn.getVertexCount('*')

In [ ]:
# Number of vertices of a specific type
conn.getVertexCount("Paper")

In [ ]:
# Number of edges for every type
conn.getEdgeCount()

In [ ]:
# Number of edges of a specific type
conn.getEdgeCount("Cite")

## Feature Engineering ##
The ML Workbench includes quite a few graph algorithms to perform feature engineering tasks. The key functions are:

1. `listAlgorithm()`: If it gets the class of algorithms (e.g. Centrality) as an input, it will print the available algorithms for    the specified category; otherwise will print the entire available algorithms. 
2. `installAlgorithm()`: Gets tha name of the algorithmm as input and installs the algorithm if it is not already installed. 
3. `runAlgorithmm()`: Gets the algorithm name, schema type (e.g. vertex/edge, by default it is vertex), attribute name (if the result needs to be stored as an attribute in the database), and a list of schema type names (list of vertices/edges that the attribute needs to be saved in, by default it is for all vertices/edges).  

In [ ]:
f = conn.gds.featurizer()

In [ ]:
f.listAlgorithms()

## Examples of running graph algorithms from GDS library ##
In the following, one example of each class of algoeirhms are provided. Some algorithms will generate a feature per vertex/edge;however, some other algorithms will calculates a number or statistics information about the graph. For example, the common neighbor algorithm calculates the number of common neighbors between two vertices.

## Get Pagerank as a feature ##
The pagerank is available in GDS library called tg_pagerank under the class of centrality algorithms https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Centrality/pagerank/global/unweighted/tg_pagerank.gsql.

In [ ]:
f.installAlgorithm("tg_pagerank")

In [ ]:
params = {'v_type': 'Paper', 'e_type': 'Cite', 'max_change': 0.001, 'max_iter': 25, 'damping': 0.85,
          'top_k': 10, 'print_accum': True, 'result_attr': '', 'file_path': '', 'display_edges': False}

f.runAlgorithm(
    'tg_pagerank', 
    params=params, 
    feat_name="pagerank",
    global_schema=False, # If schema change is needed to add an attribute, should it be a global schema change?
    timeout=2147480, 
    sizeLimit=2000000
)


## Run Maximal Independent Set ##
The Maximal Independent Set algorithm is available in GDS library called tg_maximal_indep_set under the class of classification algorithms https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Classification/maximal_independent_set/deterministic/tg_maximal_indep_set.gsql.

In [ ]:
f.installAlgorithm("tg_maximal_indep_set")

In [ ]:
params = {'v_type': 'Paper', 'e_type': 'Cite',
          'max_iter': 100, 'print_accum': False, 'file_path': ''}

f.runAlgorithm('tg_maximal_indep_set', params=params)


## Get Louvain as a feature ##
The Louvain algorithm is available in GDS library called tg_louvain under the class of community detection algorithms  https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Community/louvain/tg_louvain.gsql.

In [ ]:
f.installAlgorithm(query_name='tg_louvain')

In [ ]:
params = {'v_type': 'Paper', 'e_type': ['Cite', 'reverse_Cite'], 'wt_attr': "",
          'max_iter': 10, 'result_attr': "cid", 'file_path': "", 'print_info': True}

f.runAlgorithm(
    'tg_louvain', 
    params, 
    feat_name="cid",
    global_schema=False # If schema change is needed to add an attribute, should it be a global schema change?
)


## Get fastRP as a feature ##
The fastRP algorithm is available in GDS library called tg_fastRP under the class of community detection algorithms  https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/GraphML/Embeddings/FastRP/tg_fastRP.gsql. To install fastRP, a UDF is needed. See tutorial 2.1 UDF installer for details.

In [ ]:
f.installAlgorithm("tg_fastRP")

In [ ]:
params = {'v_type': 'Paper', 'e_type': ['Cite', 'reverse_Cite'], 'weights': '1,1,2', 'beta': -0.85, 'k': 3, 'reduced_dim': 128,
          'sampling_constant': 1, 'random_seed': 42, 'print_accum': False, 'result_attr': "", 'file_path': ""}
f.runAlgorithm(
    'tg_fastRP', 
    params, 
    feat_name="fastrp_embedding"
)


## Run Breadth-First Search Algorithm from a single source node ##
The Breadth-First Search algorithm is available in GDS library called tg_bfs under the class of Path algorithms https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Path/bfs/tg_bfs.gsql.

In [ ]:
f.installAlgorithm(query_name='tg_bfs')

In [ ]:
params = {'v_type': 'Paper', 'e_type': ['Cite', 'reverse_Cite'], 'max_hops': 10, "v_start": ("2180", "Paper"),
          'print_accum': False, 'result_attr': "", 'file_path': "", 'display_edges': False}

f.runAlgorithm('tg_bfs', params, feat_name="bfs")


## Calculates the number of common neighbors between two vertices ##
The common neighbors algorithm is available in GDS library called tg_common_neighbors under the class of Topological Link Prediction algorithms https://github.com/tigergraph/gsql-graph-algorithms/blob/master/algorithms/Topological%20Link%20Prediction/common_neighbors/tg_common_neighbors.gsql


In [ ]:
f.installAlgorithm(query_name='tg_common_neighbors')  

In [ ]:
params = {"a": ("2180", "Paper"), "b": ("431", "Paper"),
          "e_type": "Cite", "print_res": True}

f.runAlgorithm('tg_common_neighbors', params)


# Install User Defined Query

In [ ]:
user_defined_query1 = '''
CREATE QUERY user_defined_query1() FOR GRAPH Cora { 
  PRINT "user_defined_query works!"; 
}'''

print(user_defined_query1)

In [ ]:
outFileName="./user_defined_query1.gsql"
outFile=open(outFileName, "w")
outFile.write(user_defined_query1)
outFile.close()

In [ ]:
f.installAlgorithm(query_name="user_defined_query1",query_path="./user_defined_query1.gsql" )

In [ ]:
f.runAlgorithm(query_name="user_defined_query1",custom_query=True)